1. перебор сдлеать в виде функции
2. сделать вначале отправку запроса в браузер, а потом сбор json (чтобы навярняка были цены в кеше
3. после получения таблицы сделать алгоритм подбора маршрута с сортировкой от самого дешевого
4. сделать локальный файл-базу в котором будет инфа о том, что я отправлял запрос, и после каждого запуска скрипта, чтобы этот файл чистился (все что старое удалялось)

In [19]:
#!pip install selenium

In [64]:
# https://support.travelpayouts.com/hc/ru/categories/200358578-API-%D0%B8-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5
# https://support.travelpayouts.com/hc/ru/articles/203956163-Aviasales-API-%D0%B4%D0%BE%D1%81%D1%82%D1%83%D0%BF%D0%B0-%D0%BA-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC-%D0%B4%D0%BB%D1%8F-%D1%83%D1%87%D0%B0%D1%81%D1%82%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2-%D0%BF%D0%B0%D1%80%D1%82%D0%BD%D0%B5%D1%80%D1%81%D0%BA%D0%BE%D0%B9-%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D1%8B#15

In [1]:
import requests
import re
import json

from itertools import product
from tqdm import tqdm

from datetime import datetime
from datetime import timedelta

from time import sleep, strftime
import pandas as pd
from selenium import webdriver

# Используйте тут ваш путь к chromedriver!
chromedriver_path = 'chromedriver.exe'

driver = webdriver.Chrome(executable_path=chromedriver_path) # Этой командой открывается окно Chrome


add_zero = lambda x: '0' + str(x) if len(str(x)) < 2 else str(x)

def range_date(datetime_start, datetime_end):
    delta = timedelta(days = 1)
    range_dates = []
    date = datetime_start
    while date <= datetime_end:
        range_dates.append(date)
        date = date + delta
    
    return range_dates




In [2]:
import json
import codecs
f = codecs.open('airports.json', encoding='utf-8', mode='r')
data = json.load(f)


In [3]:
url = 'http://api.travelpayouts.com/data/routes.json'
response = requests.get(url)
routes = json.loads(response.text)

In [4]:
ACCESS_TOKEN = 'b8848ff67e92768d2d1977862372d796'

In [5]:
origin = 'KWE' #'KWE'
final_destination = 'LED' #'LED' 'KLO'

time_sleep = 2

destination_list = list(filter(lambda x: x['departure_airport_iata'] == origin, routes))
destination_list = list(map(lambda x: x['arrival_airport_iata'],destination_list))

date_start = '2020-01-07'
date_end = '2020-01-15'

url = 'http://api.travelpayouts.com/v2/prices/month-matrix'

datetime_start = datetime.strptime(date_start, '%Y-%m-%d')
datetime_end = datetime.strptime(date_end, '%Y-%m-%d')

tickets = []

parameters = list(product(range_date(datetime_start, datetime_end), destination_list))

for datetime_object, destination in tqdm(parameters, total=len(parameters)):
    month, day = add_zero(datetime_object.month), add_zero(datetime_object.day)
    date = datetime_object.strftime('%Y-%m-%d')
    
    url_for_cash = f'https://www.aviasales.ru/search/{origin}{day}{month}{destination}1'
    driver.get(url_for_cash)
    sleep(time_sleep)
    
for datetime_object, destination in tqdm(parameters, total=len(parameters)):
    
    params = {'currency': 'rub', 'show_to_affiliates':'false', 'token' : ACCESS_TOKEN, 'one_way':'true',
            'origin': origin, 'destination': destination, 'month':date}

    response = requests.get(url, params=params)
    
    if 'errors' not in json.loads(response.text)['data']:
        tickets = tickets + json.loads(response.text)['data']
        
        
ponts_list =  list(set(map(lambda x: x['destination'], tickets)))
parameters = list(product(range_date(datetime_start, datetime_end), ponts_list))

tickets2 = []
for datetime_object, point in tqdm(parameters, total=len(parameters)):
    month, day = add_zero(datetime_object.month), add_zero(datetime_object.day)
    date = datetime_object.strftime('%Y-%m-%d')
    
    url_for_cash = f'https://www.aviasales.ru/search/{origin}{day}{month}{destination}1'
    driver.get(url_for_cash)
    sleep(time_sleep)
    
for datetime_object, point in tqdm(parameters, total=len(parameters)):
    
    params = {'currency': 'rub', 'show_to_affiliates':'false', 'token' : ACCESS_TOKEN, 'one_way':'true',
            'origin': point, 'destination': final_destination, 'month':date}

    response = requests.get(url, params=params)
    
    if 'errors' not in json.loads(response.text)['data']:
        tickets2 = tickets2 + json.loads(response.text)['data']
        
df = pd.DataFrame(tickets + tickets2)
df.to_csv('test.txt')


100%|████████████████████████████████████████████████████████████████████████████████| 324/324 [01:16<00:00,  3.08it/s]


In [8]:
df[df['destination'] == 'LED'].sort_values(by = 'value')

,actual,depart_date,destination,distance,duration,found_at,gate,number_of_changes,origin,return_date,show_to_affiliates,trip_class,value
9922,True,2020-01-02,LED,7553,1890.0,2019-10-27T07:39:42,S7,1,HKG,None,False,0,18644.0
9723,True,2020-01-02,LED,7553,1890.0,2019-10-27T07:39:42,S7,1,HKG,None,False,0,18644.0
10625,True,2020-01-02,LED,7553,1890.0,2019-10-27T07:39:42,S7,1,HKG,None,False,0,18644.0
9269,True,2020-01-02,LED,7553,1890.0,2019-10-27T07:39:42,S7,1,HKG,None,False,0,18644.0
11057,True,2020-01-02,LED,7553,1890.0,2019-10-27T07:39:42,S7,1,HKG,None,False,0,18644.0
10852,True,2020-01-02,LED,7553,1890.0,2019-10-27T07:39:42,S7,1,HKG,None,False,0,18644.0
10171,True,2020-01-02,LED,7553,1890.0,2019-10-27T07:39:42,S7,1,HKG,None,False,0,18644.0
10398,True,2020-01-02,LED,7553,1890.0,2019-10-27T07:39:42,S7,1,HKG,None,False,0,18644.0
9496,True,2020-01-02,LED,7553,1890.0,2019-10-27T07:39:42,S7,1,HKG,None,False,0,18644.0
10971,True,2020-01-01,LED,6214,515.0,2019-10-25T09:58:48,SuperSaver.ru,0,CTU,None,False,0,19191.0
